### Shradha Yadav - U6757772

##  Booking.com Reviews

##### Questions:
 This file contains over 515,000 guest reviews and rating of almost 1500 hotels across Europe scraped from popular hotel reservation website Booking.com. The text data was cleaned by removing unicode and punctuation and transformed to lower case. No other preprocessing was done. More information on each field is provided in the "Data Description" tab of the Excel file.

Extending the analysis of this data set from class, you are asked to answer the following questions:

1. What are the top hotel features that guests talk about the most in their Booking.com reviews? Since this is a copy-and-paste from class handout, no points are assigned for this question. (0 point)
2. Score each guest review along each of the above features. Note that features that are not mentioned in a given review should receive no score for that review. You may use the approach discussed in class or a different approach of your choice.
3. Compute the average feature score for each review, and report the correlation between this average score and the “Reviewer_Score” column in the data file. A strong positive correlation (>0.70) would indicate that your scoring mechanism worked. If your correlation was not up to the mark, you may want to explore what you can do to improve it.
4. Compute average feature-wise score for each hotel (across all reviews) and report the correlation between these computed hotel scores and Average_Hotel_Score.
5. Create a dashboard with the following plots: (1) Ten highest rated hotels in Europe (showing your aggregate score and Average_Hotel_Score), (2) Ten highest rated hotels for each of the top five features identified in Question 1.
6. Regress Average_Hotel_Score against the top five feature scores of each hotel, and provide quantitative estimates of how important each feature is for the Average_Hotel_Score.


##### steps -,
    1. Read questions,
    2. Identify columns required to answer - hotel,Average_Hotel_Score, positive_revew, negative_review,Reviewer_Score ,
    3. Identify datatypes and if uniform through or not,
    4. Null and missing values in required column,
    5. no. of records in excel,
    6. Functions required -> Feature extraction,polarity/scoring, correlation, regression, plot(multiple graph),
    7. Unique column/indexing

##### Importing packages

In [ ]:

import nltk
import pandas as pd
import numpy as np
from textblob import TextBlob, Word
from string import punctuation
from collections import Counter
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.sentiment import SentimentAnalyzer
stopwords = stopwords.words('english')
nltk.download('wordnet')
nltk.download('universal_tagset')
nltk.downloader.download('vader_lexicon')
nltk.downloader.download('vader_lexicon')

import spacy
sp = spacy.load("en_core_web_sm")

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.sentiment import vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

##### Importing data file

In [ ]:
df = pd.read_excel('/Users/shradhayadav/Documents/Text_Analytics/Data/BookingDotCom_HotelReviews.xlsx')


In [ ]:
df.shape

### Data Cleaning

##### 1.droping not required column

In [ ]:
to_drop = ['Non_Review_Scoring_Count']
df.drop(to_drop, inplace=True, axis=1)
df.head()

##### 2. Changing the index of a DataFrame

In [ ]:
# df.set_index(['col'])

##### 3. check & remove missing values

In [ ]:
if df.isnull().values.any():
    # get indices
    print(df.isnull().any())
else:
    print('No Null')

In [ ]:
#Since Comments has ' No Positive' & 'No Negative' text - which means no input from customer
#thus we can remove these
df.loc[df['Positive_Review']=='No Positive''Positive_Review'] = ''
df['Positive_Review'].isnull().any()

In [ ]:
df.loc[df['Negative_Review']=='No Negative''Negative_Review'] = ''
df['Negative_Review'].isnull().any()

##### drop null if any

In [ ]:
df.dropna(subset=['Positive_Review','Negative_Review'], inplace = True)
df.shape

##### check and convert datatypes

In [ ]:
df.dtypes

##### using sample dataset to runthrough code

In [ ]:
dfs = df.sample(n=60000)
dfs.head()

##### Using the DataFrame.applymap() function to clean the entire dataset, element-wise

##### Renaming columns to a more recognizable set of labels

##### Skipping unnecessary rows in a CSV file

### Data Pre-Processing

### Feature Extraction

##### Steps,
    1. Word Tokenize all comments,
    2. Remove stopwords & Puctuations,
    3. Lemmatization,
    4. Sort nouns by their frequencies,
    5. Fetch top 15/10/5 to identify top feature

In [ ]:
def top_5(d):
        comment_list = d.tolist()                                     # Convert a Pandas series to a list
        comment_list = [c for c in comment_list if pd.isnull(c)==False]
        comment_str  = ' '.join(comment_list)                          # Concatenate all comments to one string
        tokenized = nltk.word_tokenize(comment_str)                    # Tokenize into words
        tokenized = [w for w in tokenized if len(w)>2]
        tagged = nltk.pos_tag(tokenized tagset='universal')           # Extract POS tags for each word
        bigrams = nltk.bigrams(tagged)                                 # Extract bigrams of POS tagged words
        adj_noun = [(x y) for x y in bigrams if x[1]=='ADJ' and y[1]=='NOUN']
        adj_noun = [i[0][0] + ' ' + i[1][0] for i in adj_noun]         # Convert ADJ-NOUN bigram list into a list of strings
        return nltk.FreqDist(adj_noun).most_common(5)                  # Return the 5 most common ADJ-NOUN strings


In [ ]:
def top_10_nouns(d):
    comment_list = d.tolist()
    comment_list = [c for c in comment_list if pd.isnull(c)==False]
    comment_str  = ' '.join(comment_list) 
    words = nltk.word_tokenize(comment_str) 
    words = [w for w in words if len(w)>2]
    words = [lemmatizer.lemmatize(w) for w in words]
    tagged = nltk.pos_tag(words tagset='universal')
    nouns = [w[0] for w in tagged if w[1]=='NOUN']
    return nltk.FreqDist(nouns).most_common(10)

##### Since few reviews starts with number converting datatype to string

In [ ]:
pos = dfs['Positive_Review'].str.lower()
dneg = dfs['Negative_Review'].str.lower()
    

#### Question-1 . What are the top hotel features that guests talk about the most in their Booking.com reviews? Since this is a copy-and-paste from class handout, no points are assigned for this question

In [ ]:
top_pos_features = top_5(dpos)
print(top_pos_features)

In [ ]:
top_Neg_features = top_5(dneg)
print(top_Neg_features)

In [ ]:
top_10_nouns(dpos)
top_10_nouns(dneg)

##### Question-1 Analysis/Conclusion

##### Top features :

From Positive Comments - 
From Negative Comments -

"Hotel" is not a feature of a hotel. So we will exclude it from analysis.

Combining the two sets of features top features overall that we will be using for scoring are : staff location room breakfast bed bathroom service

Note: Random sample (n=60000) gives the same features


#### Question-2. Score each guest review along each of the above features. Note that features that are not mentioned in a given review should receive no score for that review. You may use the approach discussed in class or a different approach of your choice

##### Sentence tokenize each review 
    1.Concatenating positive & Negative reviews for each reviewer by period
    
    2.Updating any word in upper to lower to keep uniform capitalize case
    
    3.Tokenizing reviews by Capital character after appending period assuming new sentence starting with Capital
    
    

In [ ]:

reviews=[]
for i in df.index:
    review = df['Positive_Review'][i]
    review = str(review).strip() 
    
    if  review.islower() or review.isupper(): 
        review = review.capitalize()
    print(review)

#     r_split = review.split()    #some sentences are upper case  capitalizing 
#     for i in range(len(r_split)):
#         if len(r_split[i])> 2 and r_split[i].isupper():
#             r_split[i] = r_split[i].lower()

#     review = ' '.join(r_split)
#     regex = re.sub( r"([A-Z]\w+[a-z])", r".\1", review)
    reviews.append(review)
# print(reviews)



In [ ]:
def comment_cleanup(comment):
    reviews=[]
    for i in df.index:
        review = comment[i]
        review = str(review).strip() 
        if  review.islower() or review.isupper(): 
            review = review.capitalize()

        r_split = review.split()    #some sentences are upper case  capitalizing 
        for i in range(len(r_split)):
            if len(r_split[i])> 2 and r_split[i].isupper():
                r_split[i] = r_split[i].lower()

        review = ' '.join(r_split)
        regex = re.sub( r"([A-Z]\w+[a-z])", r".\1", review)
        reviews.append(review)

    return reviews

In [ ]:
df_Pos = df['Positive_Review']
P_review = comment_cleanup(df_Pos)
print(P_review)

In [ ]:
df_Neg = df['Negative_Review']
N_review = comment_cleanup(df_Neg)

##### Concatenate Positive & Negative reviews and add to Dataframe df as a column 'Reviews'

In [ ]:
Reviews = [i + j for i, j in zip(P_review, N_review)]
df['Reviews'] = Reviews

In [ ]:
def sent_tokenize(Reviews):
    
    regex = re.sub( r"([A-Z]\w+[a-z])", r".\1", Reviews)
    merged_review = regex.split('.')
   
    for m in merged_review:
        if m == ' ' or m=='':
            merged_review.remove(m)

           
    for m in merged_review:
        reviews.append(m)
        for f in features:
            if f in m:
                polarity_score = []
                analyzer = SentimentIntensityAnalyzer()
                p = analyzer.polarity_scores(m)
                polarity_score = (p['compound'])

                # Recording all tokenized senetences, respective features and each sentence polarity 
                # against each reviewer in dataframe df_Score 
    
                feature_score_temp = pd.DataFrame({'d_Index':[i], 'Hotel_Name':Hotel_Name
                                                   ,'Review':review,'Review_sent':m,'Feature':f
                                                   ,'Polarity_Score': polarity_score,'Reviewer_Score': Reviewer_Score
                                                   ,'Average_Hotel_Score': Average_Hotel_Score
                                                  })
                df_Score = pd.concat([df_Score,feature_score_temp])


                                                                                        
df_Score=df_Score.drop(['d_Index'],axis=1)
df_Score.reset_index(drop=True, inplace=True) 
df_Score

In [ ]:
for i in df.index:
    print(df['Reviews'][1])

##### Function to score each review by feature

In [ ]:
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from statistics import mean

analyzer = SentimentIntensityAnalyzer()

def scoring(Reviews,feature):
    sent_list = nltk.sent_tokenize(Reviews)
    sent_list = [s for s in sent_list if re.search(feature,s) != None]
    polarity = [analyzer.polarity_scores(s)['compound'] for s in sent_list]
    if len(polarity) >=1:
        mean_polarity = mean(polarity)
    else:
        mean_polarity =0
    
    scaled_polarity = round((mean_polarity*10 + 10)/2,1)
    return scaled_polarity

In [ ]:
import time
starttime = time.time()

features = ['staff', 'location', 'room', 'bed', 'breakfast', 'bathroom']

for f in features:
    df[f] = pd.Series([scoring(r,f) for r in df['Reviews']])

print('This process took', (time.time() - starttime)/60, ' minutes')
    

In [ ]:
df.head(10)